In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import ctrgcn_zhr
import torch
from PIL import Image
import random
import json

# 随机挑选五个视频和一个rotate的视频
def chaos_frame(random_elements):
    array_dir = '单个视频的64帧特征YOLO'
    array1 = np.load('../../{}/x_tensor_origin_{}.npy'.format(array_dir, random_elements[0]))
    array2 = np.load('../../{}/x_tensor_origin_{}.npy'.format(array_dir, random_elements[1]))
    array3 = np.load('../../{}/x_tensor_origin_{}.npy'.format(array_dir, random_elements[2]))
    array4 = np.load('../../{}/x_tensor_origin_{}.npy'.format(array_dir, random_elements[3]))
    array5 = np.load('../../{}/x_tensor_origin_{}.npy'.format(array_dir, random_elements[4]))
    array1_rotate = np.load('../../{}/x_tensor_rotate_{}.npy'.format(array_dir, random_elements[0]))
    array_x = np.vstack((array2,array1,array3))
    array_y = np.vstack((array4,array1_rotate,array5))
    return array_x, array_y

# 随机分成两份或者三分,存入json文件，
def Exchange_frame(similarity_numpy, image_index):
    random_number_1 = random.randint(1, 64)

    first_three_columns = similarity_numpy[:, :random_number_1]
    last_two_columns = similarity_numpy[:, random_number_1:]
    similarity_numpy = np.hstack((last_two_columns, first_three_columns))

    
    point_left_up = [(64-random_number_1), 64]
    point_right_down = [(128-random_number_1), 128]

    label = ''
    label = label+'0'
    label = label+' '+str(((point_right_down[0]-point_left_up[0])/2+point_left_up[0])/192)
    label = label+' '+str(((point_right_down[1]-point_left_up[1])/2+point_left_up[1])/192)
    label = label+' '+str(((point_right_down[0]-point_left_up[0]))/192)
    label = label+' '+str(((point_right_down[1]-point_left_up[1]))/192)


    with open('../../目标检测txt文件反装颜色/{}.txt'.format(image_index), 'w') as file:
        file.write(label)

    return similarity_numpy



Model_zhr_temp = ctrgcn_zhr.Temporal_CNN_Model()
weights = torch.load('../../runs-zhr_try_to_train_fourth_8.pt')
Model_zhr_temp.load_state_dict(weights)

my_arrays = np.load('../../单个视频的64帧特征YOLO/my_arrays.npz')
label_numpy = my_arrays['name_numpy'].astype(int)
for zzz in range(500):
  random_elements = np.random.choice(label_numpy, size=5, replace=False)
  array_x, array_y = chaos_frame(random_elements)

  similarity_numpy = np.array([])
  for i in range(len(array_x)):
      for j in range(len(array_y)):
          norm_1 = np.linalg.norm(array_x[i])
          norm_2 = np.linalg.norm(array_y[j])
          similarity_score = np.dot(array_x[i], array_y[j]) / (norm_1*norm_2)
          similarity_numpy = np.append(similarity_numpy, similarity_score.item())
  similarity_numpy = similarity_numpy.reshape(192,192)


  similarity_numpy = Exchange_frame(similarity_numpy, zzz)

  # numpy_array = Model_zhr_temp(torch.from_numpy(similarity_numpy).unsqueeze(0).unsqueeze(1).to(torch.float32)).detach().numpy()
  # numpy_array = numpy_array[0][0]

  fig = plt.figure(figsize=(4, 4))
  plt.imshow(similarity_numpy, cmap='coolwarm', interpolation='nearest', vmin=0, vmax=1)
  plt.xticks([])
  plt.yticks([])
  plt.savefig('../../目标检测图片反装颜色/{}.png'.format(zzz), bbox_inches='tight', pad_inches=0)  # 可以指定保存的文件名和文件格式
  # plt.imshow(numpy_array, cmap='coolwarm', interpolation='nearest', vmin=0, vmax=1)
  # plt.xticks([])
  # plt.yticks([])
  # # plt.savefig('../../目标检测图片3/image_{}_copy.png'.format(zzz), bbox_inches='tight', pad_inches=0)  # 可以指定保存的文件名和文件格式

plt.title("Plot 2D array")
plt.colorbar()
plt.show()

# 打开图像文件
image = Image.open('image_0_copy.png')  # 请将'your_image.jpg'替换为您的图像文件路径

# 获取图像的尺寸
width, height = image.size

print("图像宽度:", width)
print("图像高度:", height)


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import ctrgcn_zhr
import torch
from PIL import Image
import random
import json
from scipy.ndimage import gaussian_filter

# 随机挑选五个视频和一个rotate的视频
def chaos_frame(random_elements):
    array_dir = '单个视频的64帧特征YOLO'
    array1 = np.load('../../{}/x_tensor_origin_{}.npy'.format(array_dir, random_elements[0]))
    array1_rotate = np.load('../../{}/x_tensor_rotate_{}.npy'.format(array_dir, random_elements[0]))
    random_array1 = np.random.rand(8, 6400)
    random_array2 = np.random.rand(8, 6400)
    random_array3 = np.random.rand(8, 6400)
    random_array4 = np.random.rand(8, 6400)
    # array1 = np.vstack((random_array1, array1, random_array2))
    # array1_rotate = np.vstack((random_array3, array1_rotate, random_array4))
    print(array1.shape)
    return array1, array1_rotate

# 随机分成两份或者三分,存入json文件，
def Exchange_frame(similarity_numpy, image_index):
    random_number_1 = random.randint(1, 64)

    first_three_columns = similarity_numpy[:, :random_number_1]
    last_two_columns = similarity_numpy[:, random_number_1:]
    similarity_numpy = np.hstack((last_two_columns, first_three_columns))

    
    point_left_up = [(64-random_number_1), 64]
    point_right_down = [(128-random_number_1), 128]

    label = ''
    label = label+'0'
    label = label+' '+str(((point_right_down[0]-point_left_up[0])/2+point_left_up[0])/192)
    label = label+' '+str(((point_right_down[1]-point_left_up[1])/2+point_left_up[1])/192)
    label = label+' '+str(((point_right_down[0]-point_left_up[0]))/192)
    label = label+' '+str(((point_right_down[1]-point_left_up[1]))/192)


    with open('../../目标检测txt文件YOLO2/{}.txt'.format(image_index), 'w') as file:
        file.write(label)

    return similarity_numpy



Model_zhr_temp = ctrgcn_zhr.Temporal_CNN_Model()
weights = torch.load('../../runs-zhr_try_to_train_fourth_8.pt')
Model_zhr_temp.load_state_dict(weights)

my_arrays = np.load('../../单个视频的64帧特征YOLO/my_arrays.npz')
label_numpy = my_arrays['name_numpy'].astype(int)
for zzz in range(500,1000):
  random_elements = np.random.choice(label_numpy, size=5, replace=False)
  array_x, array_y = chaos_frame(random_elements)

  similarity_numpy = np.array([])
  for i in range(len(array_x)):
      for j in range(len(array_y)):
          norm_1 = np.linalg.norm(array_x[i])
          norm_2 = np.linalg.norm(array_y[j])
          similarity_score = np.dot(array_x[i], array_y[j]) / (norm_1*norm_2)
          similarity_numpy = np.append(similarity_numpy, similarity_score.item())
  similarity_numpy = similarity_numpy.reshape(64,64)

  original_array = similarity_numpy

  # 设置要添加的圈数
  num_rings = 8

  # 对原始数组进行边界扩展，使用高斯模糊来使边界平滑
  padded_array = np.pad(original_array, num_rings, mode='edge')
  smoothed_array = gaussian_filter(padded_array, sigma=1.5)

  # 提取处理后的中间部分，去除边界扩展的圈数
  final_array = smoothed_array[num_rings:-num_rings, num_rings:-num_rings]

  # similarity_numpy[0:8,0:8] = similarity_numpy[0:8,20:28]
  # similarity_numpy[0:8,72:80] = similarity_numpy[0:8,20:28]
  # similarity_numpy[72:80,0:8] = similarity_numpy[72:80,20:28]
  # similarity_numpy[72:80,72:80] = similarity_numpy[72:80,20:28]
  
#   similarity_numpy = Exchange_frame(similarity_numpy, zzz)

  # numpy_array = Model_zhr_temp(torch.from_numpy(similarity_numpy).unsqueeze(0).unsqueeze(1).to(torch.float32)).detach().numpy()
  # numpy_array = numpy_array[0][0]

  fig = plt.figure(figsize=(4, 4))
  plt.imshow(final_array, cmap='coolwarm', interpolation='nearest', vmin=0, vmax=1)
  plt.xticks([])
  plt.yticks([])
  label = '1 0.5 0.5 0.82 0.82'
  with open('../../目标检测txt文件YOLO2/{}.txt'.format(zzz), 'w') as file:
    file.write(label)
  # plt.show()
  # plt.savefig('../../目标检测图片YOLO2/{}.png'.format(zzz), bbox_inches='tight', pad_inches=0)  # 可以指定保存的文件名和文件格式
  print(zzz)
  # plt.imshow(numpy_array, cmap='coolwarm', interpolation='nearest', vmin=0, vmax=1)
  # plt.xticks([])
  # plt.yticks([])
  # # plt.savefig('../../目标检测图片3/image_{}_copy.png'.format(zzz), bbox_inches='tight', pad_inches=0)  # 可以指定保存的文件名和文件格式




/home/zhanghaoran/miniconda3/envs/CTR_GCN_ZHR/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(64, 6400)
500
(64, 6400)
501
(64, 6400)
502
(64, 6400)
503
(64, 6400)
504
(64, 6400)
505
(64, 6400)
506
(64, 6400)
507
(64, 6400)
508
(64, 6400)
509
(64, 6400)
510
(64, 6400)
511
(64, 6400)
512
(64, 6400)
513
(64, 6400)
514
(64, 6400)
515
(64, 6400)
516
(64, 6400)
517
(64, 6400)
518
(64, 6400)
519
(64, 6400)


/home/zhanghaoran/miniconda3/envs/CTR_GCN_ZHR/lib/python3.6/site-packages/matplotlib/pyplot.py:513: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


520
(64, 6400)
521
(64, 6400)
522
(64, 6400)
523
(64, 6400)
524
(64, 6400)
525
(64, 6400)
526
(64, 6400)
527
(64, 6400)
528
(64, 6400)
529
(64, 6400)
530
(64, 6400)
531
(64, 6400)
532
(64, 6400)
533
(64, 6400)
534
(64, 6400)
535
(64, 6400)
536
(64, 6400)
537
(64, 6400)
538
(64, 6400)
539
(64, 6400)
540
(64, 6400)
541
(64, 6400)
542
(64, 6400)
543
(64, 6400)
544
(64, 6400)
545
(64, 6400)
546
(64, 6400)
547
(64, 6400)
548
(64, 6400)
549
(64, 6400)
550
(64, 6400)
551
(64, 6400)
552
(64, 6400)
553
(64, 6400)
554
(64, 6400)
555
(64, 6400)
556
(64, 6400)
557
(64, 6400)
558
(64, 6400)
559
(64, 6400)
560
(64, 6400)
561
(64, 6400)
562
(64, 6400)
563
(64, 6400)
564
(64, 6400)
565
(64, 6400)
566
(64, 6400)
567
(64, 6400)
568
(64, 6400)
569
(64, 6400)
570
(64, 6400)
571
(64, 6400)
572
(64, 6400)
573
(64, 6400)
574
(64, 6400)
575
(64, 6400)
576
(64, 6400)
577
(64, 6400)
578
(64, 6400)
579
(64, 6400)
580
(64, 6400)
581
(64, 6400)
582
(64, 6400)
583
(64, 6400)
584
(64, 6400)
585
(64, 6400)
586
(64, 6

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import ctrgcn_zhr
import torch
import math

kkk = np.load('../../单个视频的64帧特征10/x_tensor_092439_numpy_1.npy')

print(kkk.shape)

t1 = 0
t2 = 200

# 尝试只用其中的一些骨骼点，比如引体向上不用腿部骨骼点特征
ZZZ = np.load('../../单个视频的64帧特征10/x_tensor_092439_numpy_2(用我的模型跑的).npy')
ZZZ = ZZZ.reshape(2880, 25, 256)
# ZZZ = np.delete(ZZZ, [0,1,2,3,4,5,6,7,8,9,10,11,19,20,21,22,23,24,    12, 13, 14, 16, 17, 18], axis=1)
# ZZZ = np.delete(ZZZ, [ 13, 14, 15, 17, 18,19], axis=1)
ZZZ = ZZZ.reshape(2880,6400)
# print(ZZZ.shape)

similarity_numpy = np.array([])
for i in range(40,62,1):
    for j in range(t1,t2):
        norm_1 = np.linalg.norm(ZZZ[i])
        norm_2 = np.linalg.norm(ZZZ[j])
        similarity_score = np.dot(ZZZ[i], ZZZ[j]) / (norm_1*norm_2)
        similarity_numpy = np.append(similarity_numpy, similarity_score.item())
# for i in range(len(similarity_numpy)):
#     similarity_numpy[i] = 1.0 - similarity_numpy[i]
similarity_numpy_2 = similarity_numpy.reshape(22,200)

array = np.random.uniform(0.65, 0.8, size=(10, 200))

similarity_numpy_2 = np.vstack((array,similarity_numpy_2,array))

for i in range(5):
    fig = plt.figure(figsize=(4, 4))
    plt.imshow(similarity_numpy_2[:,i*40:(i+1)*40], cmap='coolwarm', interpolation='nearest',vmin=0,vmax=1) # [:,i*40:(i+1)*40]
    plt.xticks([])
    plt.yticks([])
    # plt.savefig('heatmap_try_{}.png'.format(i), bbox_inches='tight', pad_inches=0)
    # plt.title("Plot 2D array")
    # plt.colorbar()
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import ctrgcn_zhr
import torch
from PIL import Image
import random
import json

# 随机挑选五个视频和一个rotate的视频
def chaos_frame(random_elements):
    array_dir = '单个视频的64帧特征YOLO'
    array1 = np.load('../../{}/x_tensor_origin_{}.npy'.format(array_dir, random_elements[0]))
    array1_rotate = np.load('../../{}/x_tensor_rotate_{}.npy'.format(array_dir, random_elements[0]))
    return array1, array1_rotate

# 随机分成两份或者三分,存入json文件，
def Exchange_frame(similarity_numpy, image_index):
    random_number_1 = random.randint(1, 64)

    first_three_columns = similarity_numpy[:, :random_number_1]
    last_two_columns = similarity_numpy[:, random_number_1:]
    similarity_numpy = np.hstack((last_two_columns, first_three_columns))

    
    point_left_up = [(64-random_number_1), 64]
    point_right_down = [(128-random_number_1), 128]

    label = ''
    label = label+'0'
    label = label+' '+str(((point_right_down[0]-point_left_up[0])/2+point_left_up[0])/192)
    label = label+' '+str(((point_right_down[1]-point_left_up[1])/2+point_left_up[1])/192)
    label = label+' '+str(((point_right_down[0]-point_left_up[0]))/192)
    label = label+' '+str(((point_right_down[1]-point_left_up[1]))/192)


    with open('../../目标检测txt文件反装颜色/{}.txt'.format(image_index), 'w') as file:
        file.write(label)

    return similarity_numpy



Model_zhr_temp = ctrgcn_zhr.Temporal_CNN_Model()
weights = torch.load('../../runs-zhr_try_to_train_fourth_8.pt')
Model_zhr_temp.load_state_dict(weights)

my_arrays = np.load('../../单个视频的64帧特征YOLO/my_arrays.npz')
label_numpy = my_arrays['name_numpy'].astype(int)
for zzz in range(10):
  random_elements = np.random.choice(label_numpy, size=5, replace=False)
  array_x, array_y = chaos_frame(random_elements)

  similarity_numpy = np.array([])
  for i in range(len(array_x)):
      for j in range(len(array_y)):
          norm_1 = np.linalg.norm(array_x[i])
          norm_2 = np.linalg.norm(array_y[j])
          similarity_score = np.dot(array_x[i], array_y[j]) / (norm_1*norm_2)
          similarity_numpy = np.append(similarity_numpy, similarity_score.item())
  similarity_numpy = similarity_numpy.reshape(64,64)
#   similarity_numpy = Exchange_frame(similarity_numpy, zzz)

  # numpy_array = Model_zhr_temp(torch.from_numpy(similarity_numpy).unsqueeze(0).unsqueeze(1).to(torch.float32)).detach().numpy()
  # numpy_array = numpy_array[0][0]

  fig = plt.figure(figsize=(4, 4))
  plt.imshow(similarity_numpy, cmap='coolwarm', interpolation='nearest', vmin=0, vmax=1)
  plt.xticks([])
  plt.yticks([])
  plt.show()
#   plt.savefig('../../目标检测图片反装颜色/{}.png'.format(zzz), bbox_inches='tight', pad_inches=0)  # 可以指定保存的文件名和文件格式
  # plt.imshow(numpy_array, cmap='coolwarm', interpolation='nearest', vmin=0, vmax=1)
  # plt.xticks([])
  # plt.yticks([])
  # # plt.savefig('../../目标检测图片3/image_{}_copy.png'.format(zzz), bbox_inches='tight', pad_inches=0)  # 可以指定保存的文件名和文件格式



In [39]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import ctrgcn_zhr
import torch
from PIL import Image
import random
import json

# 随机挑选五个视频和一个rotate的视频
def chaos_frame(random_elements):
    array_dir = '单个视频的64帧特征YOLO'
    array1 = np.load('../../{}/x_tensor_origin_{}.npy'.format(array_dir, random_elements[0]))
    array2 = np.load('../../{}/x_tensor_origin_{}.npy'.format(array_dir, random_elements[1]))
    array3 = np.load('../../{}/x_tensor_origin_{}.npy'.format(array_dir, random_elements[2]))
    array4 = np.load('../../{}/x_tensor_origin_{}.npy'.format(array_dir, random_elements[3]))
    array5 = np.load('../../{}/x_tensor_origin_{}.npy'.format(array_dir, random_elements[4]))
    array1_rotate = np.load('../../{}/x_tensor_rotate_{}.npy'.format(array_dir, random_elements[0]))
    array_x = np.vstack((array2,array1,array3))
    array_y = np.vstack((array4,array1_rotate,array5))
    return array_x, array_y

# 随机分成两份或者三分,存入json文件，
def Exchange_frame(similarity_numpy, image_index):
    random_number_1 = random.randint(1, 64)

    first_three_columns = similarity_numpy[:, :random_number_1]
    last_two_columns = similarity_numpy[:, random_number_1:]
    similarity_numpy = np.hstack((last_two_columns, first_three_columns))

    
    point_left_up = [(64-random_number_1), 64]
    point_right_down = [(128-random_number_1), 128]




    return similarity_numpy



Model_zhr_temp = ctrgcn_zhr.Temporal_CNN_Model()
weights = torch.load('../../runs-zhr_try_to_train_fourth_8.pt')
Model_zhr_temp.load_state_dict(weights)

my_arrays = np.load('../../单个视频的64帧特征YOLO/my_arrays.npz')
label_numpy = my_arrays['name_numpy'].astype(int)
for zzz in range(100):
  print(zzz)
  random_elements = np.random.choice(label_numpy, size=5, replace=False)
  array_x, array_y = chaos_frame(random_elements)

  array_x = array_x[54:138]
  # array_y = array_y[40:168]
  array_y = array_y[44:148]

  similarity_numpy = np.array([])
  for i in range(len(array_x)):
      for j in range(len(array_y)):
          norm_1 = np.linalg.norm(array_x[i])
          norm_2 = np.linalg.norm(array_y[j])
          similarity_score = np.dot(array_x[i], array_y[j]) / (norm_1*norm_2)
          similarity_numpy = np.append(similarity_numpy, similarity_score.item()-0.05)
  similarity_numpy = similarity_numpy.reshape(len(array_x),len(array_y))

  # similarity_numpy = similarity_numpy[54:138,40:168]


#   similarity_numpy = Exchange_frame(similarity_numpy, zzz)

  # numpy_array = Model_zhr_temp(torch.from_numpy(similarity_numpy).unsqueeze(0).unsqueeze(1).to(torch.float32)).detach().numpy()
  # numpy_array = numpy_array[0][0]

  fig = plt.figure(figsize=(5, 5))
  plt.imshow(similarity_numpy, cmap='coolwarm', interpolation='nearest', vmin=0, vmax=1)
  plt.xticks([])
  plt.yticks([])
  plt.savefig('../../YOLO/datasets/valid/images/{}.png'.format(zzz), bbox_inches='tight', pad_inches=0)  # 可以指定保存的文件名和文件格式
  label = '0 0.5 0.5 0.61538 0.7619'
  with open('../../YOLO/datasets/valid/labels/{}.txt'.format(zzz), 'w') as file:
    file.write(label)
  # plt.imshow(numpy_array, cmap='coolwarm', interpolation='nearest', vmin=0, vmax=1)
  # plt.xticks([])
  # plt.yticks([])
  # # plt.savefig('../../目标检测图片3/image_{}_copy.png'.format(zzz), bbox_inches='tight', pad_inches=0)  # 可以指定保存的文件名和文件格式
  # plt.show()




